#판례 데이터로 도전하는 파이썬 자연어처리
1차: soynlp: Unsupervised tokenizers in NLP project

##1차: soynlp: Unsupervised tokenizers in NLP project

---

1.   Install NLP Tools & Libraries
2.   항목 추가



###Install NLP Tools & Libraries

*   konlpy 설치방법: http://konlpy.org/en/latest/install/
*   mecab 설치방법: https://calyfactory.github.io/mecab%EC%9D%98-custom-dictionary%EC%99%80-konlpy-%EC%97%B0%EB%8F%99%ED%95%98%EA%B8%B0/



In [0]:
!pip3 install sqlalchemy pymysql
!pip3 install soynlp # soyspacing soykeyword

In [0]:
import os

#Konlpy 설치 (feat.JPype1-py3)
!apt-get update
!apt-get install g++ openjdk-8-jdk python-dev python3-dev
!pip3 install JPype1-py3
!pip3 install konlpy
!JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"

#폴더 만들기
!mkdir mecab 

#mecab-ko 설치
os.chdir('/content/mecab')
!wget "https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz"
!tar zxfv mecab-0.996-ko-0.9.2.tar.gz
!rm -rf mecab-0.996-ko-0.9.2.tar.gz
os.chdir('./mecab-0.996-ko-0.9.2')
!./configure
!make
!make check
!make install

#mecab-ko-dic 설치
os.chdir('/content/mecab')
!wget "https://bitbucket.org/eunjeon/mecab-ko-dic/downloads/mecab-ko-dic-2.0.3-20170922.tar.gz"
!tar zxfv mecab-ko-dic-2.0.3-20170922.tar.gz
!rm -rf mecab-ko-dic-2.0.3-20170922.tar.gz
os.chdir('./mecab-ko-dic-2.0.3-20170922')
!apt-get install automake1.11
!./autogen.sh
!./configure
!ldconfig
!make
!make install

#mecab-python3 설치
!git clone https://bitbucket.org/eunjeon/mecab-python-0.996.git
os.chdir('/content/mecab/mecab-python-0.996/')
!python3 setup.py build
!python3 setup.py install
os.chdir('/content/mecab')

# --- Command "python setup.py egg_info" failed with error code 1 in /tmp/pip-install-2pigm5yo/mecab-python3/ 으로 인해 수정됨 ---
# os.chdir('/content/mecab')
# !apt install aptitude
# !aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
# !pip install mecab-python3

In [0]:
# konlpy와 mecab의 설치여부 확인
print('CWD:', os.getcwd())
print('Mecab:', os.listdir('/content/mecab'))
print('Mecab_userdic:', os.listdir('/content/mecab/mecab-ko-dic-2.0.3-20170922/user-dic'))

CWD: /content/mecab
Mecab: ['mecab-python-0.996', 'mecab-ko-dic-2.0.3-20170922', 'mecab-0.996-ko-0.9.2']
Mecab_userdic: ['README.md', 'person.csv', 'place.csv', 'nnp.csv']


In [0]:
# konlpy와 mecab의 작동여부 확인
from konlpy.tag import Mecab
mecab = Mecab()
print(mecab.morphs(u'영등포구청역에 있는 맛집 좀 알려주세요.'))
['영등포구', '청역', '에', '있', '는', '맛집', '좀', '알려', '주', '세요', '.']
print(mecab.nouns(u'우리나라에는 무릎 치료를 잘하는 정형외과가 없는가!'))
['우리', '나라', '무릎', '치료', '정형외과']
print(mecab.pos(u'자연주의 쇼핑몰은 어떤 곳인가?'))
[('자연', 'NNG'), ('주', 'NNG'), ('의', 'JKG'), ('쇼핑몰', 'NNG'), ('은', 'JX'), ('어떤', 'MM'), ('곳', 'NNG'), ('인가', 'VCP+EF'), ('?', 'SF')]

['영등포구청역', '에', '있', '는', '맛집', '좀', '알려', '주', '세요', '.']
['우리', '나라', '무릎', '치료', '정형외과']
[('자연주의', 'NNG'), ('쇼핑몰', 'NNG'), ('은', 'JX'), ('어떤', 'MM'), ('곳', 'NNG'), ('인가', 'VCP+EF'), ('?', 'SF')]


[('자연', 'NNG'),
 ('주', 'NNG'),
 ('의', 'JKG'),
 ('쇼핑몰', 'NNG'),
 ('은', 'JX'),
 ('어떤', 'MM'),
 ('곳', 'NNG'),
 ('인가', 'VCP+EF'),
 ('?', 'SF')]

###Import Python Libraries & Modules

In [0]:
%matplotlib inline
# import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# nltk.sent_tokenize() 사용을 위한 punkt 패키지 다운로드
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
#-- coding:utf-8 --
from contextlib import contextmanager

class MySQLConnectnDataframe:

    def __init__(self, userName, userPassword, hostServer, dbName):
        from sqlalchemy import create_engine
        from sqlalchemy.pool import NullPool
        import pymysql
        pymysql.install_as_MySQLdb()
        import MySQLdb
        self.engine = create_engine(
                            'mysql+mysqldb://'
                            + userName + ':'
                            + userPassword + '@'
                            + hostServer + '/'
                            + dbName
                            + '?charset=utf8', encoding='utf8',
                            poolclass=NullPool
        )

    def _getDataframeFromQueryOnMySqlTable(self, query):
        from pandas import read_sql
        conn = self.engine.connect()
        print('00 MySQL Database is connected.')
        df = read_sql(query, con=conn)
        print('00 %i Records has been queried to Dataframe.' % len(df) )
        conn.close()
        print('00 MySQL Database is disconnected.')
        return(df)

###Query openlawdb.casetbl Table From DB

사건번호(casenumber) 의미와 종류: https://namu.wiki/w/%EC%82%AC%EA%B1%B4%EB%B2%88%ED%98%B8

In [0]:
db = MySQLConnectnDataframe('userName','userPassword','hostServer','dbName')
casetbl = db._getDataframeFromQueryOnMySqlTable("select * from `openlawdb`.`casetbl` limit 0, 10000;")

00 MySQL Database is connected.
00 10000 Records has been queried to Dataframe.
00 MySQL Database is disconnected.


In [0]:
# openlawdb.casetbl 맛보기(1)
casetbl[casetbl['caseserialnumber'] == casetbl['caseserialnumber'][5878]]
# casetbl[casetbl['casenumber'].str.contains('나|다|라|마|그|바')].head(1)

,caseserialnumber,casename,casenumber,sentencedate,sentence,courtname,courttypecode,casetype,casetypecode,judgmenttype,judgment,decisionpoint,referenceprovision,referencecase,casecontent
5878,72712,폭력행위등처벌에관한법률위반피고사건,70노99,1970-04-28,,서울고법,400202,,400102,제1형사부판결 : 확정,항소는 하지 아니하고 상고장만 제출한 후 항소취하를 한 경우의 재판관할,항소는 하지 아니하고 상고장만 제출한 후 항소취하서를 제출한 경우에는 위의 항소취하...,"형사소송법 제359조\n, \n 제349조\n, \n 제319조","1953.4.21. 선고 4285형상116 판결(판례카아드 8822호, 판결요지집 ...",【항 소 인】 피고인\n\n판사 이종진(재판장) 이재인 문진탁


In [0]:
# openlawdb.casetbl 맛보기(2)
from pprint import pprint
pprint(casetbl['casecontent'][104])

('【피 고 인】 \n'
 '【상 고 인】 피고인\n'
 '【변 호 인】 변호사 김정기\n'
 '【원심판결】 대전지법 2008. 8. 22. 선고 2007노3284 판결\n'
 '【주    문】\n'
 '  원심판결을 파기하고, 사건을 대전지방법원 본원 합의부에 환송한다.\n'
 '\n'
 '【이    유】  직권으로 판단한다.\n'
 '제1심은 피고인을 \n'
 '          구 소년법(2007. 12. 21. 법률 제8722호로 개정되기 전의 것) 제2조에 의한 소년으로 인정하여 \n'
 '          위 구 소년법 제60조 제1항에 의하여 부정기형을 선고하였고, 이에 대하여 피고인이 양형부당을 이유로 항소하였으나 '
 '원심은 피고인의 항소를 기각하여 제1심판결을 유지하였다.  그런데 위와 같이 법률 제8722호로 개정된 소년법이 \n'
 '          같은 법 부칙 제1조에 정한 대로 2008. 6. 22.부터 시행된바, \n'
 '          개정 소년법 제2조에 의하여 ‘소년’의 정의가 ‘20세 미만’에서 ‘19세 미만’으로 개정되었고, 그 규정은 \n'
 '          같은 법 부칙 제2조에 따라 같은 법 시행 당시 심리중에 있는 형사사건인 이 사건에 관하여도 적용되며, 한편 피고인은 '
 '1989. 2. 10.생으로 원심판결 선고일인 2008. 8. 22.에 이미 19세에 달하였음이 기록상 명백하다. 결국, 피고인은 \n'
 '          개정 소년법 제2조의 소년에 해당하지 않으므로, 피고인에 대하여 부정기형을 선고한 제1심판결을 파기하고 정기형을 '
 '선고하는 조치를 취하지 아니한 원심에는 \n'
 '          소년법 제2조의 소년에 관한 법리를 오해하여 판결에 영향을 미친 위법이 있다.\n'
 '        \n'
 '  그러므로 상고이유에 대한 판단을 생략한 채 원심판결을 파기하고, 사건을 다시 심리·판단하게 하기 위하여 원심법원에 환송하기로 하여 '
 '관여 대법관의 일치된 의견으로 주문과 같

###Explore openlawdb.casetbl Data

In [0]:
# '【'를 마지막에 추가해 소제목과 내용을 분리해내도록 한다.
casecontent = casetbl['casecontent'].str[:]+'【'

In [0]:
# Column과 Row의 개수가 매칭되는지 확인하며 CrossCheck!
columnDict = {}
import re
inside = re.compile(r'\【(.*?)\】')
outside = re.compile(r'\】(.*?)\【', re.DOTALL)

Error = []
print("START Investigate Correctness for Sperating 'column':row' of casecontent")
for i, content in enumerate(casecontent):
    
    if i % 1000 == 0:
        print('-'*5, i, 'th casecontent', '-'*5)
    
    chap = inside.findall(content)
    para = outside.findall(content)
    
    if len(chap) == len(para) is False:
        print('CrossCheck for Speration: ', len(chap) == len(para))
        Error.append(i)
    else:
        if not len(chap) in columnDict:
            columnDict[len(chap)] = [i]
        else:
            columnDict[len(chap)].append(i)
        
print('End Investigation (Error count: %s)' % (len(Error)))

START Investigate Correctness for Sperating 'column':row' of casecontent
----- 0 th casecontent -----
----- 1000 th casecontent -----
----- 2000 th casecontent -----
----- 3000 th casecontent -----
----- 4000 th casecontent -----
----- 5000 th casecontent -----
----- 6000 th casecontent -----
----- 7000 th casecontent -----
----- 8000 th casecontent -----
----- 9000 th casecontent -----
End Investigation (Error count: 0)


In [0]:
# casecontent에서 column:row 쌍의 현황 파악
print("[Distribution of 'column':'row' SET in casecontent]")
for key in columnDict.keys():
    print('%s쌍: %s개' % (key, len(columnDict[key])))

[Distribution of 'column':'row' SET in casecontent]
7쌍: 759개
5쌍: 3943개
6쌍: 4691개
3쌍: 58개
11쌍: 20개
8쌍: 165개
4쌍: 207개
9쌍: 101개
10쌍: 37개
12쌍: 14개
13쌍: 1개
1쌍: 1개
2쌍: 3개


In [0]:
casecontent[5878]

'【항 소 인】   피고인\n\n판사 이종진(재판장) 이재인 문진탁【'

In [0]:
# column 예시
import re
inside = re.compile(r'\【(.*?)\】')
chap = inside.findall(casecontent[104])
chap = [re.sub('\s+', ' ', c).strip() for c in chap]
print(chap)


inside = re.compile(r'\【(.*?)\】')
chap = inside.findall(casecontent[3000])
chap = [re.sub('\s+', ' ', c).strip() for c in chap]
print(chap)

['피 고 인', '상 고 인', '변 호 인', '원심판결', '주 문', '이 유']
['원고', '피고', '변론종결', '주문', '청구취지', '이유']


In [0]:
# row 예시
outside = re.compile(r'\】(.*?)\【', re.DOTALL)
para = outside.findall(casecontent[104])
para = [re.sub('\s+', ' ', p).strip() for p in para]
print(para)

['', '피고인', '변호사 김정기', '대전지법 2008. 8. 22. 선고 2007노3284 판결', '원심판결을 파기하고, 사건을 대전지방법원 본원 합의부에 환송한다.', '직권으로 판단한다. 제1심은 피고인을 구 소년법(2007. 12. 21. 법률 제8722호로 개정되기 전의 것) 제2조에 의한 소년으로 인정하여 위 구 소년법 제60조 제1항에 의하여 부정기형을 선고하였고, 이에 대하여 피고인이 양형부당을 이유로 항소하였으나 원심은 피고인의 항소를 기각하여 제1심판결을 유지하였다. 그런데 위와 같이 법률 제8722호로 개정된 소년법이 같은 법 부칙 제1조에 정한 대로 2008. 6. 22.부터 시행된바, 개정 소년법 제2조에 의하여 ‘소년’의 정의가 ‘20세 미만’에서 ‘19세 미만’으로 개정되었고, 그 규정은 같은 법 부칙 제2조에 따라 같은 법 시행 당시 심리중에 있는 형사사건인 이 사건에 관하여도 적용되며, 한편 피고인은 1989. 2. 10.생으로 원심판결 선고일인 2008. 8. 22.에 이미 19세에 달하였음이 기록상 명백하다. 결국, 피고인은 개정 소년법 제2조의 소년에 해당하지 않으므로, 피고인에 대하여 부정기형을 선고한 제1심판결을 파기하고 정기형을 선고하는 조치를 취하지 아니한 원심에는 소년법 제2조의 소년에 관한 법리를 오해하여 판결에 영향을 미친 위법이 있다. 그러므로 상고이유에 대한 판단을 생략한 채 원심판결을 파기하고, 사건을 다시 심리·판단하게 하기 위하여 원심법원에 환송하기로 하여 관여 대법관의 일치된 의견으로 주문과 같이 판결한다. 대법관 김영란(재판장) 이홍훈 안대희(주심) 양창수']


In [0]:
# 실제 유의미한 문장 단위를 얻어내기 위한 전처리
DMiddle = re.compile(r'(\d{2,4})\.(\s+)(\d{1,2})\.(\s+|.)')
DEnd = re.compile(r'\-(\d{1,2})\.')
D3Dots = re.compile(r'(\d{2,4})\.(\d{1,2})\.(\d{1,2})\.')
YM = re.compile(r'(\d{2,4})\-(\d{1,2})\-')
YMD = re.compile(r'(\d{2,4})\-(\d{1,2})\-(\d{1,2})')
Comma = re.compile(r'\,(\d{3})')
Mark = re.compile(r'(\s)(\w)(\.)')

dateMiddle = DMiddle.sub(r"\g<1>-\g<3>-", para[-1])
# dateMiddle = DMiddle.sub(r"\g<1>-\g<3>-", '2011. 12. 1')
dateEnd = DEnd.sub(r"-\g<1>", dateMiddle)
dateDot2Dash = D3Dots.sub(r"\g<1>-\g<2>-\g<3>", dateEnd)
dateYMD = YMD.sub(r"\g<1>년\g<2>월\g<3>일", dateDot2Dash)
dateYM = YM.sub(r"\g<1>년\g<2>월 ", dateYMD)
moneyComma = Comma.sub(r"\g<1>", dateYM)
contentMark = Mark.sub(r'.\g<1>\g<2>.', moneyComma).replace('..', '.')
for bracket in re.findall('\.[\"\'\>\]\}\)]', contentMark):
    contentMark = contentMark.replace(bracket, bracket[-1])

In [0]:
nltk.sent_tokenize(contentMark)

['직권으로 판단한다.',
 '제1심은 피고인을 구 소년법(2007년12월21일 법률 제8722호로 개정되기 전의 것) 제2조에 의한 소년으로 인정하여 위 구 소년법 제60조 제1항에 의하여 부정기형을 선고하였고, 이에 대하여 피고인이 양형부당을 이유로 항소하였으나 원심은 피고인의 항소를 기각하여 제1심판결을 유지하였다.',
 '그런데 위와 같이 법률 제8722호로 개정된 소년법이 같은 법 부칙 제1조에 정한 대로 2008년6월22일부터 시행된바, 개정 소년법 제2조에 의하여 ‘소년’의 정의가 ‘20세 미만’에서 ‘19세 미만’으로 개정되었고, 그 규정은 같은 법 부칙 제2조에 따라 같은 법 시행 당시 심리중에 있는 형사사건인 이 사건에 관하여도 적용되며, 한편 피고인은 1989년2월10일생으로 원심판결 선고일인 2008년8월22일에 이미 19세에 달하였음이 기록상 명백하다.',
 '결국, 피고인은 개정 소년법 제2조의 소년에 해당하지 않으므로, 피고인에 대하여 부정기형을 선고한 제1심판결을 파기하고 정기형을 선고하는 조치를 취하지 아니한 원심에는 소년법 제2조의 소년에 관한 법리를 오해하여 판결에 영향을 미친 위법이 있다.',
 '그러므로 상고이유에 대한 판단을 생략한 채 원심판결을 파기하고, 사건을 다시 심리·판단하게 하기 위하여 원심법원에 환송하기로 하여 관여 대법관의 일치된 의견으로 주문과 같이 판결한다.',
 '대법관 김영란(재판장) 이홍훈 안대희(주심) 양창수']

In [0]:
def casecontent_to_dict(content):
    inside, outside = re.compile(r'\【(.*?)\】'), re.compile(r'\】(.*?)\【', re.DOTALL)
    columns = [re.sub('\s+', ' ', column).strip() for column in inside.findall(content)] 
    rows = [re.sub('\s+', ' ', row).strip() for row in outside.findall(content)] 
    
    DMiddle = re.compile(r'(\d{2,4})\.(\s+)(\d{1,2})\.(\s+|.)')
    DEnd = re.compile(r'\-(\d{1,2})\.')
    D3Dots = re.compile(r'(\d{2,4})\.(\d{1,2})\.(\d{1,2})\.')
    YM = re.compile(r'(\d{2,4})\-(\d{1,2})\-')
    YMD = re.compile(r'(\d{2,4})\-(\d{1,2})\-(\d{1,2})')
    Comma = re.compile(r'\,(\d{3})')
    Mark = re.compile(r'(\s)(\w)(\.)')
    
    Dict = {}
    for col, row in zip(columns, rows):
        dateMiddle = DMiddle.sub(r"\g<1>-\g<3>-", row)
        dateEnd = DEnd.sub(r"-\g<1>", dateMiddle)
        dateDot2Dash = D3Dots.sub(r"\g<1>-\g<2>-\g<3>", dateEnd)
        dateYMD = YMD.sub(r"\g<1>년\g<2>월\g<3>일", dateDot2Dash)
        dateYM = YM.sub(r"\g<1>년\g<2>월 ", dateYMD)
        moneyComma = Comma.sub(r"\g<1>", dateYM)
        contentMark = Mark.sub(r'.\g<1>\g<2>.', moneyComma).replace('..', '.')
        for bracket in re.findall('\.[\"\'\>\]\}\)]', contentMark):
            contentMark = contentMark.replace(bracket, bracket[-1])
        Dict[col] = contentMark
        
    return Dict

casecontent_to_dict(casecontent[11])

{'원 고': '',
 '원고승계참가인, 피상고인': '',
 '원심판결': '서울고법 2007년7월24일 선고 2006나49682 판결',
 '이 유': '직권으로 살펴본다. 1. 채권에 대한 압류 및 추심명령이 있으면 제3채무자에 대한 이행의 소는 추심채권자만이 제기할 수 있고 채무자는 피압류채권에 대한 이행소송을 제기할 당사자적격을 상실한다고 하여야 할 것이다( 대법원 2000년4월11일 선고 99다23888 판결 등 참조). 그리고 위와 같은 당사자적격에 관한 사항은 소송요건에 관한 것으로서 사실심의 변론종결시를 기준으로 법원이 이를 직권으로 조사하여 판단하여야 하고( 대법원 1994년9월30일 선고 93다27703 판결 등 참조), 비록 당사자가 사실심 변론종결시까지 이에 관하여 주장하지 아니하였다고 하더라도 상고심에서 새로이 이를 주장, 입증할 수 있다고 할 것이다( 대법원 1989년10월10일 선고 89누1308 판결 등 참조). 2. 피고의 상고이유보충서(상고이유서 제출기간을 도과하여 제출된 것으로서, 법원의 직권 발동을 촉구하는 의미의 범위 내에서 살펴본다)의 주장 및 기록에 의하면, 원고는 이 사건 매매계약의 해제로 인한 계약금반환채권 및 손해배상채권(이하 통칭하여 ‘이 사건 채권’이라고 한다)에 기하여 피고를 상대로 이 사건 청구를 한 사실, 원심 소송계속 중이던 2006년4월25일 원고가 이 사건 채권을 원고 승계참가인에게 양도하고 같은 날 위 채권양도 사실을 피고에게 통지한 사실, 그런데 위 채권양도가 이루어지기 전인 2006년3월8일 원고의 채권자인 최광윤이 원고의 피고에 대한 이 사건 채권에 관하여 채권압류 및 추심명령( 대전지방법원 공주지원 2006타채70)을 받았고 위 채권압류 및 추심명령이 2006년3월15일 제3채무자인 피고에게 송달된 사실을 알 수 있다. 3. 앞서 본 바와 같은 법리에 기초하여 위와 같은 사실들 및 기록에 비추어 살펴보면, 2006년3월15일 이 사건 채권에 관한 위 채권압류 및 추심명령이 제3채무자인 피고

In [0]:
nltk.sent_tokenize(casecontent_to_dict(casecontent[11])['이 유'])

['직권으로 살펴본다.',
 '1.',
 '채권에 대한 압류 및 추심명령이 있으면 제3채무자에 대한 이행의 소는 추심채권자만이 제기할 수 있고 채무자는 피압류채권에 대한 이행소송을 제기할 당사자적격을 상실한다고 하여야 할 것이다( 대법원 2000년4월11일 선고 99다23888 판결 등 참조).',
 '그리고 위와 같은 당사자적격에 관한 사항은 소송요건에 관한 것으로서 사실심의 변론종결시를 기준으로 법원이 이를 직권으로 조사하여 판단하여야 하고( 대법원 1994년9월30일 선고 93다27703 판결 등 참조), 비록 당사자가 사실심 변론종결시까지 이에 관하여 주장하지 아니하였다고 하더라도 상고심에서 새로이 이를 주장, 입증할 수 있다고 할 것이다( 대법원 1989년10월10일 선고 89누1308 판결 등 참조).',
 '2.',
 '피고의 상고이유보충서(상고이유서 제출기간을 도과하여 제출된 것으로서, 법원의 직권 발동을 촉구하는 의미의 범위 내에서 살펴본다)의 주장 및 기록에 의하면, 원고는 이 사건 매매계약의 해제로 인한 계약금반환채권 및 손해배상채권(이하 통칭하여 ‘이 사건 채권’이라고 한다)에 기하여 피고를 상대로 이 사건 청구를 한 사실, 원심 소송계속 중이던 2006년4월25일 원고가 이 사건 채권을 원고 승계참가인에게 양도하고 같은 날 위 채권양도 사실을 피고에게 통지한 사실, 그런데 위 채권양도가 이루어지기 전인 2006년3월8일 원고의 채권자인 최광윤이 원고의 피고에 대한 이 사건 채권에 관하여 채권압류 및 추심명령( 대전지방법원 공주지원 2006타채70)을 받았고 위 채권압류 및 추심명령이 2006년3월15일 제3채무자인 피고에게 송달된 사실을 알 수 있다.',
 '3.',
 '앞서 본 바와 같은 법리에 기초하여 위와 같은 사실들 및 기록에 비추어 살펴보면, 2006년3월15일 이 사건 채권에 관한 위 채권압류 및 추심명령이 제3채무자인 피고에게 송달됨으로써 이 사건 채권에 기한 이행의 소는 추심채권자만이 제기할 수 있게 되고, 원고는 이 사건

###Preprocessing openlawdb.casetbl Data

In [0]:
new_columns = {'이유':'decision_reason', '재판관':'judge_list'}

casetbl = pd.concat([casetbl, pd.DataFrame(columns=new_columns.values())])
casetbl['casecontent'] = casetbl['casecontent'].str[:]+'【'

In [0]:
print("START Classifying 'decision_reason' & 'judge_list' FROM 'casecontent'")
t_start = time.time()
t_epoch = t_start
for idx, row in casetbl.iterrows():
    if idx % 1000 == 0:
        _time = time.time()
        print('-'*5, idx, 'th casecontent is finished: %02d:%02d' % divmod(_time-t_epoch, 60),  '-'*5)
        t_epoch = _time
        
    content_to_dict = casecontent_to_dict(row['casecontent'])
    content_to_dict = { re.sub('\s+', '', key):value for key, value in content_to_dict.items() }
    
    column_sorter = filter(lambda x: x == '이유', (key for key in content_to_dict.keys()))
    for col in column_sorter:
        doc = content_to_dict[col]
        sent = nltk.sent_tokenize(doc)
        try:
            judge_list = sent[-1]
            judge_list = re.sub('\s+',' ', judge_list)
            sep = re.findall('대법관 |판사 ', judge_list)[-1]
            s_index = re.search(sep, judge_list).start() 
            _sent = judge_list[:s_index].strip()+'.' if len(judge_list[:s_index]) != 0 else '' 
            judge_list = judge_list[s_index:]
            
            if len(re.findall('대법관 |판사 ', judge_list)) > 0:
                casetbl.loc[casetbl['caseserialnumber'].isin([row['caseserialnumber']]), new_columns[col]] = ' '.join(sent[:-1] + [_sent]).strip()        
                casetbl.loc[casetbl['caseserialnumber'].isin([row['caseserialnumber']]), 'judge_list'] = judge_list
            else:
                casetbl.loc[casetbl['caseserialnumber'].isin([row['caseserialnumber']]), new_columns[col]] = ' '.join(sent)        
                casetbl.loc[casetbl['caseserialnumber'].isin([row['caseserialnumber']]), 'judge_list'] = np.nan
        except IndexError: # '', 빈칸인 경우
            casetbl.loc[casetbl['caseserialnumber'].isin([row['caseserialnumber']]), new_columns.values()] = np.nan
        
t_end = time.time()
print("END Classifying: total %02d:%02d" % divmod(t_end-t_start, 60))   

START Classifying 'decision_reason' & 'judge_list' FROM 'casecontent'
----- 0 th casecontent is finished: 00:00 -----
----- 1000 th casecontent is finished: 00:07 -----
----- 2000 th casecontent is finished: 00:07 -----
----- 3000 th casecontent is finished: 00:07 -----
----- 4000 th casecontent is finished: 00:10 -----
----- 5000 th casecontent is finished: 00:09 -----
----- 6000 th casecontent is finished: 00:07 -----
----- 7000 th casecontent is finished: 00:08 -----
----- 8000 th casecontent is finished: 00:08 -----
----- 9000 th casecontent is finished: 00:08 -----
END Classifying: total 01:25


In [0]:
print(casetbl['decision_reason'][1])
print(casetbl['judgment'][1])

상고이유를 판단한다. 1. 상고이유 제1점에 대하여 공범인 공동피고인은 당해 소송절차에서는 피고인의 지위에 있으므로 다른 공동피고인에 대한 공소사실에 관하여 증인이 될 수 없으나, 소송절차가 분리되어 피고인의 지위에서 벗어나게 되면 다른 공동피고인에 대한 공소사실에 관하여 증인이 될 수 있다 할 것이다( 대법원 1999년9월17일 선고 99도2449 판결, 대법원 2007년11월29일 선고 2007도2661 판결 등 참조). 원심이 이와 달리, 공동피고인은 자신에 대한 공소사실과 밀접한 관련이 있는 공범인 다른 공동피고인에 대한 공소사실에 관하여는 변론의 분리 여부와 관계 없이 증인적격이 없음을 전제로, 피고인이 수원지방법원 성남지원 2007고단1674 게임산업진흥에 관한 법률 위반 사건에서 공범인 박오석에 대한 공소사실에 관하여 증인으로 출석하여 선서한 다음 증언함에 있어 기억에 반하는 허위의 진술을 하였다고 하더라도 위증죄가 성립하지 아니한다고 판단한 것은 잘못이라 할 것이다. 그러나 원심이 적법하게 채택하여 조사한 증거들에 의하면, 피고인은 위 성남지원 2007고단1674호 사건에서 “피고인은 게임장 종업원, 박오석은 게임장 운영자로서 공모하여 관할관청의 허가를 받지 않고 게임장 영업행위를 하였다.”는 게임산업진흥에 관한 법률 위반의 공소사실로 박오석과 공동으로 기소되어 심리가 진행되고 있어 피고인의 지위에 있음에도 불구하고, 박오석과 피고인의 변론이 분리되지 아니한 상태에서 박오석에 대한 공소사실에 관하여 증인으로 채택되어 선서하고 증언한 사실을 알 수 있는바, 위 법리에 비추어 보면, 피고인과 박오석의 변론이 분리되지 아니한 이상 피고인은 공범인 박오석에 대한 공소사실에 관하여 증인이 될 수 없고, 따라서 피고인이 박오석에 대한 공소사실에 관하여 증인으로 출석하여 선서한 다음 증언함에 있어 기억에 반하는 허위의 진술을 하였다고 하더라도 위증죄가 성립하지 아니한다 할 것이므로, 원심이 이 사건 위증의 공소사실을 무죄로 인정한 조치는 결과적으로 정당하다 

In [0]:
casetbl[['decision_reason','judge_list']]

,decision_reason,judge_list
0,"1. 제1심판결의 인용 당원이 이 사건에 관하여 설시할 이유는, 제1심판결 제6면 ...",판사 이인복(재판장) 이두형 박영재
1,상고이유를 판단한다. 1. 상고이유 제1점에 대하여 공범인 공동피고인은 당해 소송절...,대법관 김지형(재판장) 고현철 전수안 차한성(주심)
2,"재항고이유를 본다. 민사소송 등 인지법 제1조 본문은 민사소송절차, 행정소송절차 등...",대법관 김지형(재판장) 고현철 전수안 차한성(주심)
3,상고이유를 판단한다. 1. 원고의 상고이유에 대하여 원심판결 이유에 의하면 원심은 ...,대법관 김능환(재판장) 양승태 박시환(주심) 박일환
4,각 상고이유를 판단한다. 1. 이 사건 본소 중 임차보증금반환청구에 관한 부분 및 ...,대법관 김영란(재판장) 이홍훈 안대희(주심) 양창수
5,상고이유(상고이유서 제출기간이 경과한 후에 제출된 상고이유보충서의 기재는 상고이유를...,대법관 안대희(재판장) 김영란(주심) 이홍훈 양창수
6,상고이유를 판단한다. 1. 피고 명의의 소유권이전등기가 실체관계에 부합하는 등기라는...,대법관 박일환(재판장) 양승태(주심) 박시환 김능환
7,상고이유를 본다. 집행관이 구 민사소송법(2002년1월26일 법률 제6626호로 전...,대법관 고현철(재판장) 김지형 전수안(주심) 차한성
8,상고이유를 판단한다. 1. 시설물철거 및 토지인도 청구에 관하여 권리행사가 권리의 ...,대법관 김영란(재판장) 이홍훈 안대희(주심) 양창수
9,상고이유에 대한 판단에 앞서 직권으로 판단한다. 1. 행정처분이 당연무효라고 하기 ...,대법관 전수안(재판장) 고현철(주심) 김지형 차한성


In [0]:
casetbl.loc[casetbl['judge_list'].str.len() > 30, ['decision_reason','judge_list']]

,decision_reason,judge_list
5647,"(1) 원고는, 본소 청구원인 사실로서 1953.7.경부터 피고소유인 원성군 흥업면...",판사 조규대(재판장) 이정우 오성환 (판사) 이정우는 전근으로 서명날인 불능임)
5720,소외 서상오가 1968년12월27일에 부산지방법원 통영지원에서 수산업법시행령위반 피...,판사 이존웅(재판장) 고정권 박영도 박영도전근으로서명불능 이존웅(재판장)
5757,"피고가 원고에 대하여 그 소유의 부산 동래구 거제동 840 대 6187평, 같은동 ...",판사 이존웅(재판장) 이규진 권영목 권영목전출로서명불능 이존웅(재판장)
6003,1. 서울민사지방법원이 위 당사자간의 65카9160 부동산처분금지등 가처분신청사건에...,판사 김용철(재판장) 강현태 노종상 노종상은전출로인하여서명불능이므로 김용철(재판장)


In [0]:
casetbl.loc[casetbl['judge_list'].str.len() < 20]

,casecontent,casename,casenumber,caseserialnumber,casetype,casetypecode,courtname,courttypecode,decision_reason,decisionpoint,judge_list,judgment,judgmenttype,referencecase,referenceprovision,sentence,sentencedate
0,"【원고,항소인】 파산자 주식회사 충남상호신용금고의 파산관재인 예금보험공사 (소송...",사해행위취소등,2003나1746,64437.0,,400101,대전고법,400202,"1. 제1심판결의 인용 당원이 이 사건에 관하여 설시할 이유는, 제1심판결 제6면 ...",파산법이 파산관재인에게 파산재단에 관한 소에 있어 원고 또는 피고가 된다고 한 것은...,판사 이인복(재판장) 이두형 박영재,파산자의 채권에 기한 사해행위취소의 소에서 사해행위 사실을 알았는지 여부를 파산관재...,판결: 상고,"대법원 1990. 11. 13. 선고 88다카26987 판결(공1991, 57)","민법 제406조 제2항\n,\n\n 파산법 제7조",선고,2003-07-10
45,"【원고, 항소인 겸 피항소인】 \n【피고, 피항소인 겸 항소인】 \n【제1심판결】 ...",구상금,2005나96315,64495.0,,400101,서울고등법원,400202,"1. 기초사실 다음의 각 사실은 당사자 사이에 다툼이 없거나, 갑 2 내지 8호증,...",,판사 주기동(재판장) 최재혁 구회근,,판결,,,선고,2006-07-05
46,"【원고(탈퇴)】 \n【승계참가인, 피항소인겸항소인】 \n【피고, 항소인겸피항소인】 ...",손해배상(기),2003나41434,64496.0,,400101,서울고등법원,400202,"1. 기초사실 아래 각 사실은 당사자 사이에 다툼이 없거나, 갑제 1, 2호증 갑제...",,판사 최병덕(재판장) 조휴옥 장준현,,판결,,,선고,2006-02-08
47,"【원고, 항소인】 세계국선도연맹외 2(소송대리인 법무법인 세양 담당변호사 박병휴외 ...",인터넷도메인이름사용금지등,2005나23034,64497.0,,400101,서울고등법원,400202,"1. 기초사실 다음 사실은 당사자 사이에 다툼이 없거나, 갑 1호증의 1, 2, 갑...",,판사 주기동(재판장) 최재혁 구회근,,판결,,,선고,2006-05-24
48,"【원고, 항소인】 \n【피고, 피항소인】 \n【제1심판결】 서울중앙지방법원 2005...",한글인터넷주소사용중지등,2005나96124,64498.0,,400101,서울고등법원,400202,"1. 기초사실 다음 사실은 당사자 사이에 다툼이 없거나, 갑 1호증, 갑 4호증, ...",,판사 조용호(재판장) 김환수 김운호,,판결,,,선고,2006-06-27
49,"【원고, 항소인】 \n【피고, 피항소인】 \n【제1심판결】 수원지방법원 2006. ...",이사회결의무효확인,2006나37047,64499.0,,400101,서울고등법원,400202,1. 기초사실 이 부분에 관하여는 제1심 판결의 해당부분(제2쪽 8째줄부터 제3쪽 ...,,판사 이인복(재판장) 박재현 김성대,,판결,,,선고,2007-01-16
50,"【원고, 항소인】 신용보증기금(소송대리인 변호사 정광진)\n【피고, 피항소인】 \n...",사해행위취소,2005나1581,64500.0,,400101,서울중앙지방법원,400202,"1. 기초사실 이 법원이 이 부분에 관하여 설시할 이유는, 제1심 판결문 제2쪽 아...",,판사 임수식(재판장) 홍성욱 김민정,,판결,,,선고,2007-08-23
51,"【원고, 피항소인】 황학구역주택재개발조합(소송대리인 변호사 고영소)\n【피고, 항소...",부당이득금반환,2007나42268,64501.0,,400101,서울고등법원,400202,"1. 인정사실 당원이 이 부분에 설시할 이유는, 제1심 판결 이유 제1항 부분의 ‘...",,판사 강영호(재판장) 유진현 고종영,,판결,,,선고,2008-04-24
52,"【원고, 항소인 겸 피항소인】 \n【피고, 피항소인 겸 항소인】 \n【제1심판결】 ...",손해배상금,2007나13008,64502.0,,400101,부산고등법원,400202,"1. 인정사실 이 법원이 이 사건에 관하여 설시할 이유는, 제1심 판결의 이유 부분...",,판사 임시규(재판장) 이진수 전지환,,판결,,,선고,2008-05-14
53,"【원고, 항소인】 \n【피고, 피항소인】 농업협동조합중앙회\n【제1심판결】 수원지방...",배당이의,2007나62989,64503.0,,400101,서울고등법원,400202,"이 법원의 판결이유는 제1심 판결문 이유 기재와 같으므로, 민사소송법 제420조 본...",,판사 정현수(재판장) 박광우 김태의,,판결,,,선고,2008-05-23


In [0]:
nltk.sent_tokenize(casetbl['decision_reason'][11])

['직권으로 살펴본다.',
 '1.',
 '채권에 대한 압류 및 추심명령이 있으면 제3채무자에 대한 이행의 소는 추심채권자만이 제기할 수 있고 채무자는 피압류채권에 대한 이행소송을 제기할 당사자적격을 상실한다고 하여야 할 것이다( 대법원 2000년4월11일 선고 99다23888 판결 등 참조).',
 '그리고 위와 같은 당사자적격에 관한 사항은 소송요건에 관한 것으로서 사실심의 변론종결시를 기준으로 법원이 이를 직권으로 조사하여 판단하여야 하고( 대법원 1994년9월30일 선고 93다27703 판결 등 참조), 비록 당사자가 사실심 변론종결시까지 이에 관하여 주장하지 아니하였다고 하더라도 상고심에서 새로이 이를 주장, 입증할 수 있다고 할 것이다( 대법원 1989년10월10일 선고 89누1308 판결 등 참조).',
 '2.',
 '피고의 상고이유보충서(상고이유서 제출기간을 도과하여 제출된 것으로서, 법원의 직권 발동을 촉구하는 의미의 범위 내에서 살펴본다)의 주장 및 기록에 의하면, 원고는 이 사건 매매계약의 해제로 인한 계약금반환채권 및 손해배상채권(이하 통칭하여 ‘이 사건 채권’이라고 한다)에 기하여 피고를 상대로 이 사건 청구를 한 사실, 원심 소송계속 중이던 2006년4월25일 원고가 이 사건 채권을 원고 승계참가인에게 양도하고 같은 날 위 채권양도 사실을 피고에게 통지한 사실, 그런데 위 채권양도가 이루어지기 전인 2006년3월8일 원고의 채권자인 최광윤이 원고의 피고에 대한 이 사건 채권에 관하여 채권압류 및 추심명령( 대전지방법원 공주지원 2006타채70)을 받았고 위 채권압류 및 추심명령이 2006년3월15일 제3채무자인 피고에게 송달된 사실을 알 수 있다.',
 '3.',
 '앞서 본 바와 같은 법리에 기초하여 위와 같은 사실들 및 기록에 비추어 살펴보면, 2006년3월15일 이 사건 채권에 관한 위 채권압류 및 추심명령이 제3채무자인 피고에게 송달됨으로써 이 사건 채권에 기한 이행의 소는 추심채권자만이 제기할 수 있게 되고, 원고는 이 사건

###Sentence Tokenizing & Noun Extracting with soynlp

soynlp: https://github.com/lovit/soynlp

soynlp.word.WordExtractor: https://github.com/lovit/soynlp/blob/master/tutorials/wordextractor_lecture.ipynb

In [0]:
class Sentences:
    def __init__(self, DataFrame, column):
        self.DataFrame = DataFrame
        self.column = column
        self.length = 0
    def __iter__(self):
        for idx, doc in self.DataFrame.iterrows():
            for sent in nltk.sent_tokenize(doc[self.column]):
                yield sent
    def __len__(self):
        if self.length == 0:
            self.DataFrame[self.column] = self.DataFrame[self.column].fillna('')
            for idx, doc in self.DataFrame.iterrows():
                self.length += len(nltk.sent_tokenize(doc[self.column]))
        return self.length
    
    
sentences = Sentences(casetbl, 'decision_reason')
print('num sentences = %d' % len(sentences))

num sentences = 159774


In [0]:
%%time

from soynlp.word import WordExtractor

# word_extractor = WordExtractor(min_frequency=10)

word_extractor = WordExtractor(
    min_frequency=100,
    min_cohesion_forward=0.05, 
    min_right_branching_entropy=0.0
)

word_extractor.train(sentences)
words = word_extractor.extract()

training was done. used memory 4.668 Gb
all cohesion probabilities was computed. # words = 24887
all branching entropies was computed # words = 586510
all accessor variety was computed # words = 586510
CPU times: user 2min, sys: 2.3 s, total: 2min 2s
Wall time: 2min 2s


In [0]:
len(words)

17541

In [0]:
print('type: %s\n' % type(words['원심판결']))
print(words['원심판결'])

type: <class 'soynlp.word._word.Scores'>

Scores(cohesion_forward=0.33944979357999777, cohesion_backward=0, left_branching_entropy=2.840448783303548, right_branching_entropy=2.247145305593839, left_accessor_variety=82, right_accessor_variety=35, leftside_frequency=7490, rightside_frequency=0)


In [0]:
import math

def word_score(score):
    return (score.cohesion_forward * math.exp(score.right_branching_entropy))

print('단어   (빈도수, cohesion, branching entropy)\n')
for word, score in sorted(words.items(), key=lambda x:word_score(x[1]), reverse=True)[:30]:
    print('%s     (%d, %.3f, %.3f)' % (
            word, 
            score.leftside_frequency, 
            score.cohesion_forward,
            score.right_branching_entropy
    ))

단어   (빈도수, cohesion, branching entropy)

또는     (19297, 0.770, 5.016)
까지     (446, 0.722, 4.700)
혹은     (604, 0.910, 4.397)
걸쳐     (1849, 0.698, 4.468)
방법으로     (2243, 0.537, 4.689)
새로운     (1333, 0.662, 4.467)
의하여     (23913, 0.543, 4.579)
같은     (48912, 0.596, 4.468)
통하여     (2580, 0.432, 4.700)
곧바로     (451, 0.965, 3.884)
역시     (3116, 0.736, 4.078)
스스로     (1088, 0.623, 4.205)
때에는     (3629, 0.550, 4.299)
쪽으로     (107, 0.650, 4.104)
모두     (8859, 0.540, 4.256)
참조).     (3399, 0.631, 4.078)
경우에는     (6423, 0.481, 4.341)
일반적으로     (875, 0.392, 4.529)
함으로써     (922, 0.450, 4.377)
대하여     (32986, 0.493, 4.285)
위하여     (10558, 0.380, 4.530)
대한     (44732, 0.329, 4.662)
많은     (736, 0.500, 4.233)
과정에서     (1060, 0.396, 4.440)
함께     (2818, 0.278, 4.782)
내용의     (3412, 0.319, 4.643)
회사의     (3878, 0.421, 4.366)
사건     (82050, 0.366, 4.477)
피고인이     (5894, 0.331, 4.537)
먼저     (2698, 0.986, 3.421)


In [0]:
print(nltk.sent_tokenize(casetbl['decision_reason'][104])[-3])

그런데 위와 같이 법률 제8722호로 개정된 소년법이 같은 법 부칙 제1조에 정한 대로 2008년6월22일부터 시행된바, 개정 소년법 제2조에 의하여 ‘소년’의 정의가 ‘20세 미만’에서 ‘19세 미만’으로 개정되었고, 그 규정은 같은 법 부칙 제2조에 따라 같은 법 시행 당시 심리중에 있는 형사사건인 이 사건에 관하여도 적용되며, 한편 피고인은 1989년2월10일생으로 원심판결 선고일인 2008년8월22일에 이미 19세에 달하였음이 기록상 명백하다.


In [0]:
from konlpy.tag import Mecab

mecab = Mecab()
print(mecab.morphs(nltk.sent_tokenize(casetbl['decision_reason'][104])[-3]))

['그런데', '위', '와', '같이', '법률', '제', '8722', '호', '로', '개정', '된', '소년법', '이', '같', '은', '법', '부칙', '제', '1', '조', '에', '정한', '대로', '2008', '년', '6', '월', '22', '일', '부터', '시행', '된', '바', ',', '개정', '소년법', '제', '2', '조', '에', '의하', '여', '‘', '소년', '’', '의', '정의', '가', '‘', '20', '세', '미만', '’', '에서', '‘', '19', '세', '미만', '’', '으로', '개정', '되', '었', '고', ',', '그', '규정', '은', '같', '은', '법', '부칙', '제', '2', '조', '에', '따라', '같', '은', '법', '시행', '당시', '심리', '중', '에', '있', '는', '형사', '사건', '인', '이', '사건', '에', '관하', '여도', '적용', '되', '며', ',', '한편', '피고인', '은', '1989', '년', '2', '월', '10', '일', '생', '으로', '원심', '판결', '선고', '일', '인', '2008', '년', '8', '월', '22', '일', '에', '이미', '19', '세', '에', '달하', '였음이', '기록', '상', '명백', '하', '다', '.']


In [0]:
from soynlp.tokenizer import LTokenizer

scores = {w:s.cohesion_forward for w, s in words.items()}
tokenizer = LTokenizer(scores=scores)
print(tokenizer.tokenize(nltk.sent_tokenize(casetbl['decision_reason'][104])[-3]))

['그런데', '위와', '같이', '법률', '제8', '722호로', '개정', '된', '소년', '법이', '같은', '법', '부칙', '제1', '조에', '정한', '대로', '200', '8년6월22일부터', '시행', '된바,', '개정', '소년', '법', '제2', '조에', '의하여', '‘소', '년’의', '정의', '가', '‘20', '세', '미만', '’에서', '‘1', '9세', '미만', '’으로', '개정되', '었고,', '그', '규정', '은', '같은', '법', '부칙', '제2', '조에', '따라', '같은', '법', '시행', '당시', '심리', '중에', '있는', '형사', '사건인', '이', '사건', '에', '관하여', '도', '적용', '되며,', '한편', '피고', '인은', '19', '89년2월10일생으로', '원심판결', '선고', '일인', '200', '8년8월22일에', '이미', '19', '세에', '달하', '였음이', '기록상', '명백하다.']


In [0]:
from soynlp.tokenizer import MaxScoreTokenizer

scores = {w:s.cohesion_forward for w, s in words.items()}
tokenizer = MaxScoreTokenizer(scores=scores)
print(tokenizer.tokenize(nltk.sent_tokenize(casetbl['decision_reason'][104])[-3]))

['그런데', '위와', '같이', '법률', '제8', '72', '2호', '로', '개정', '된', '소년', '법이', '같은', '법', '부칙', '제1', '조에', '정한', '대로', '200', '8년', '6월', '22', '일부', '터', '시행', '된', '바,', '개정', '소년', '법', '제2', '조에', '의하여', '‘소', '년’의', '정의', '가', '‘', '20', '세', '미만', '’에서', '‘', '19', '세', '미만', '’으로', '개정되', '었고,', '그', '규정', '은', '같은', '법', '부칙', '제2', '조에', '따라', '같은', '법', '시행', '당시', '심리', '중에', '있는', '형사', '사건', '인', '이', '사건', '에', '관하여', '도', '적용', '되며,', '한편', '피고', '인은', '19', '89', '년', '2월', '10', '일생으로', '원심', '판결', '선고', '일인', '200', '8년', '8월', '22', '일에', '이미', '19', '세에', '달', '하였', '음이', '기록상', '명백하다.']


In [0]:
from soynlp.noun import LRNounExtractor_v2
noun_extractor = LRNounExtractor_v2(verbose=True, extract_compound=True)

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=1260, neg=1173, common=12


In [0]:
%%time

nouns = noun_extractor.train_extract(sentences)
# nouns = noun_extractor.train_extract(sentences, min_count=1, minimum_noun_score=0.3)
print('num nouns = %d' % len(nouns))

[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 813279 from 159774 sents. mem=4.243 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. mem=4.498 Gb
[Noun Extractor] batch prediction was completed for 339350 words
[Noun Extractor] checked compounds. discovered 3984 compounds
[Noun Extractor] postprocessing detaching_features : 112250 -> 111375
[Noun Extractor] postprocessing ignore_features : 111375 -> 111293
[Noun Extractor] postprocessing ignore_NJ : 111293 -> 110753
[Noun Extractor] 110753 nouns (3984 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=4.732 Gb                    
[Noun Extractor] 76.14 % eojeols are covered
num nouns = 110753
CPU times: user 1min 51s, sys: 1.4 s, total: 1min 52s
Wall time: 1min 52s


In [0]:
nouns['계약금반환채권']

NounScore(frequency=4, score=0.8)

In [0]:
# list(noun_extractor._compounds_components.items())[:50]
compounds = []
for c in list(noun_extractor._compounds_components.items())[:200]:
    if len(re.findall('1|2|3|4|5|6|7|8|9', c[0])) == 0:
        compounds.append(c)
        
compounds

[('광산개발저지투쟁위원', ('광산개발', '저지', '투쟁', '위원')),
 ('비품감가상각비명세서', ('비품', '감가상각비', '명세서')),
 ('서울특별시지방공무원', ('서울특별시', '지방공무원')),
 ('한국기술금융주식회사', ('한국', '기술금융', '주식회사')),
 ('친일반민족행위자재산', ('친일반민족행위자', '재산')),
 ('최고의사결정기관', ('최고', '의사결정기관')),
 ('특수공무집행방해치상', ('특수', '공무집행방해', '치상')),
 ('아니하였으므로(채권', ('아니', '하였으므로(', '채권')),
 ('울산북부신용협동조합', ('울산', '북부', '신용협동조합')),
 ('선원근로자재해보장책', ('선원', '근로자', '재해', '보장책')),
 ('수능등급구분점수정보', ('수능', '등급구분점수정보')),
 ('중앙환경분쟁조정위원', ('중앙', '환경', '분쟁조정', '위원')),
 ('임금구조기본통계조사', ('임금', '구조', '기본', '통계', '조사')),
 ('직영용역노무지급명세', ('직영', '용역', '노무', '지급', '명세')),
 ('교원소청심사특별위원', ('교원', '소청심사', '특별', '위원')),
 ('전국주택매매가격상승', ('전국', '주택', '매매가격', '상승')),
 ('한화국토개발주식회사', ('한화', '국토', '개발주식회사')),
 ('육군인사소청심사위원', ('육군', '인사', '소청심사', '위원')),
 ('기업재무구조개선작업', ('기업', '재무구조개선', '작업')),
 ('국민건강보험요양급여', ('국민건강보험', '요양급여')),
 ("자녀학비보조수당'이", ('자녀', '학비보조수당', "'이")),
 ('국민주택융자기금운용', ('국민주택', '융자', '기금', '운용')),
 ('대구남부신용협동조합', ('대구', '남부', '신용협동조합')),
 ('한국컨테이너부두운영', ('한국', '컨테이너', '부두', 